# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you find **at least two datasets** on the web which are related, and that you visualize these datasets to answer the assignment question. You are free to utilize datasets with any location or domain, the usage of **Ann Arbor sports and athletics** datasets in the example is just a suggestion.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * You must state a question you are seeking to answer with your visualizations.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together as an example! [Example Solution File](./readonly/Assignment4_example.pdf)

## Chicago Weather and Crime Pattern

#### Fetching Crime Data from City of Chicago Data Portal's Socrata Platform.

In [ ]:
from typing import Optional, List
from sodapy import Socrata
from pathlib import Path
from time import sleep
import requests
import csv


def write_results(data: List[dict], filepath: str, number: int) -> str:
    """
    Function used internally by other functions to write csv files.
    """
    base_path = Path(filepath)
    new_filename = base_path.stem + "_" + str(number) + base_path.suffix
    new_filepath = base_path.parent / new_filename
    with open(new_filepath, mode="w", encoding="utf-8") as outfile:
        writer = csv.writer(outfile)
        writer.writerow(data[10].keys())
        writer.writerows([item.values() for item in data])
    return new_filepath


def save_data_as_csv(
    dataset_id: str,
    app_token: str,
    limit: int = 10**5,
    offset: int = 0,
    sleep_t: int = 15,
    filepath: str = "fetched_data/save.csv",
    columns: Optional[List[str]] = None,
) -> List[list]:
    """
    Fetch data from the City of Chicago API and saves it in batches.

    Args:
        dataset_id (str): The dataset identifier.
        app_token (str): The application token for API access.
        limit (int): Number of records per page (default: 1000,000).
        offset (int): Will fetch data starting from the defined offset index (default is 0).
        sleep_t (int): How long (in seconds) to sleep between page requests.
        filepath (str): Path to the file, will be appended with offset point number.
        columns (Optional[List[str]]): A list of columns to be included in data.

    Returns:
        list[dict]: A list of JSON objects containing the crime data.
    """
    domain = "data.cityofchicago.org"
    client = Socrata(domain=domain, app_token=app_token)

    if columns:
        columns = ", ".join([col for col in columns])

    temp_data = []
    all_data = []
    break_counter = 0
    while True:
        try:
            response = client.get(
                dataset_identifier=dataset_id,
                limit=limit,
                offset=offset,
                content_type="json",
                select=columns
            )
            temp_data.extend(response)
            offset = offset + limit
            break_counter += 1
            print(f"The first {offset} entries successfully fetched.")
            if break_counter >= 10:
                # Write data if more than 10 requests run.
                new_filepath = write_results(
                    data=temp_data, filepath=filepath, number=offset
                )
                print(
                    f"Total of {offset:,} entries were fetched. Results saved in {new_filepath}"
                )
                all_data.extend(temp_data)
                temp_data = []
                break_counter = 0
            sleep(sleep_t)
        except requests.exceptions.ReadTimeout:
            print(f"Read timeout occured, will try after {sleep_t} seconds.")
            sleep(sleep_t)
            continue
        except Exception as e:
            if temp_data:
                new_filepath = write_results(
                    data=temp_data, filepath=filepath, number=offset
                )
                print(f"Some fetched data saved in {new_filepath}.\n{e}")
            print(f"Error occured {e}")
            break
    return all_data

#### Query and Save Crime Data

To query and save crime data you can start with setting the `first_page` parameter to 0 (or leave it blank), pick a reasonable `n_pages` parameter (e.g. query through 500 pages takes about 25 mins). Run and save the results recursively each time starting at the previous `first_page + n_pages` until there is no more results.

*To query all data without using any filters takes a significant amount of time (north of 4 hours)*

In [ ]:
from sodapy import Socrata
from dotenv import load_dotenv
import os

starting_offset = 6*(10**6)

load_dotenv()
app_token = os.getenv("APP_TOKEN")
dataset_id = "ijzp-q8t2"
columns = [
    "id",
    "case_number",
    "date",
    "block",
    "iucr",
    "primary_type",
    "description",
    "location_description",
    "arrest",
    "domestic",
    "beat",
    "district",
    "ward",
    "community_area",
    "fbi_code",
    "latitude",
    "longitude",
]

results = save_data_as_csv(
    dataset_id=dataset_id,
    app_token=app_token,
    limit=10**5,
    offset=starting_offset,
    filepath="data/crimes.csv",
    columns=columns,
)

In [ ]:
print(len(results))
results[10]